# Natural Language Processing
## Assignment 3 - Machine Translation
### Code 2 - With Attention
### Students
- M Maheeth Reddy (1801CS31)
- Nischal A (1801CS33)

### Import Block

In [ ]:
from io import open
import random
import math
import time
import os
import re
import sys

import string
from string import digits

from pickle import dump, load

import unicodedata
from unicodedata import normalize

from __future__ import unicode_literals, print_function, division


import numpy as np
from numpy import array, argmax
from numpy.random import rand, shuffle
import pandas as pd
import matplotlib.pyplot as plt


from nltk.translate.bleu_score import corpus_bleu, sentence_bleu


import sklearn
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split


import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F


import scipy
import statsmodels

In [ ]:
MAX_LENGTH = 100


In [ ]:
# !wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1iljWhk1LVNEdXOYyfxCZd_Osv9D5TJ76' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1iljWhk1LVNEdXOYyfxCZd_Osv9D5TJ76" -O assign4dataset.zip && rm -rf /tmp/cookies.txt
# ! unzip assign4dataset.zip

--2022-04-07 11:41:09--  https://docs.google.com/uc?export=download&confirm=&id=1iljWhk1LVNEdXOYyfxCZd_Osv9D5TJ76
Resolving docs.google.com (docs.google.com)... 74.125.195.113, 74.125.195.101, 74.125.195.102, ...
Connecting to docs.google.com (docs.google.com)|74.125.195.113|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0o-bs-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/grtkss6ei1b9t30bh5ht3ql56mbs17c2/1649331600000/15072518060054569691/*/1iljWhk1LVNEdXOYyfxCZd_Osv9D5TJ76?e=download [following]
--2022-04-07 11:41:10--  https://doc-0o-bs-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/grtkss6ei1b9t30bh5ht3ql56mbs17c2/1649331600000/15072518060054569691/*/1iljWhk1LVNEdXOYyfxCZd_Osv9D5TJ76?e=download
Resolving doc-0o-bs-docs.googleusercontent.com (doc-0o-bs-docs.googleusercontent.com)... 74.125.197.132, 2607:f8b0:400e:c03::84
Connecting to doc-0o-bs-docs.googleusercontent.com (doc-0o-bs-

In [ ]:
token_SOS = 0
token_EOS = 1


class Lang:
    def __init__(self, name):
        self.name = name
        self.word_to_index = {"sos": 0, "eos": 1}
        self.word_to_count = {}
        self.index_to_word = {0: "sos", 1: "eos"}
        self.n_words = 2  # Count SOS and EOS

    def addWord(self, word):
        if word not in self.word_to_index:
            self.word_to_index[word] = self.n_words
            self.word_to_count[word] = 1
            self.index_to_word[self.n_words] = word
            self.n_words += 1
        else:
            self.word_to_count[word] += 1

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)


In [ ]:
def eng_sent_preprocessor(sent):
    sent = sent.lower()
    sent = clean_txt(sent)
    re_print = re.compile('[^%s]' % re.escape(string.printable))
    sent = normalize('NFD', sent).encode('ascii', 'ignore')
    sent = sent.decode('UTF-8')
    sent = sent.split()
    sent = [re_print.sub('', w) for w in sent]
    sent = [word for word in sent if word.isalpha()]
    sent = ' '.join(sent)
    return sent


def hin_sent_preprocessor(sent):
    sent = re.sub('[a-zA-Z]', '', sent)
    sent = clean_txt(sent)
    exclude = set(string.punctuation)
    remove_digits = str.maketrans('', '', string.digits)
    sent = re.sub("'", '', sent)
    sent = ''.join(ch for ch in sent if ch not in exclude)
    sent = sent.translate(remove_digits)
    sent = re.sub("[२३०८१५७९४६]", "", sent)
    sent = sent.strip()
    sent = re.sub(" +", " ", sent)
    return (sent)


In [ ]:
def doc_reader(filename):
    with open(filename, mode='rt', encoding='utf-8') as file:
        text = file.read()
    return text


def make_pairs(eng_txt, hin_txt):
    eng_lines = eng_txt.strip().split('\n')
    hin_lines = hin_txt.strip().split('\n')

    pairs = []
    for i in range(len(hin_lines)):
        pairs.append([])
        pairs[i].append(eng_sent_preprocessor(eng_lines[i]))
        pairs[i].append(hin_sent_preprocessor(hin_lines[i]))
    return pairs


In [ ]:
def clean_txt(txt):
    txt = txt.replace(u',', '').replace(u'"', '').replace(u'"', '').replace(u"‘‘", '').replace(u"’’", '').replace(u"''", '').replace(u"।", '').replace(u',', '').replace(u'"', '').replace(u'(', '').replace(u')', '').replace(u'"', '').replace(
        u':', '').replace(u"'", '').replace(u"‘‘", '').replace(u"’’", '').replace(u"''", '').replace(u".", '').replace(u"-", '').replace(u"।", '').replace(u"?", '').replace(u"\\", '').replace(u"_", '').replace("'", "").replace('"', "")
    txt = re.sub("'", '', txt)
    txt = re.sub("’", '', txt)
    txt = re.sub('[0-9+\-*/.%]', '', txt)
    txt = txt.strip()
    txt = re.sub(' +', ' ', txt)
    txt = ''.join(ch for ch in txt if ch not in set(string.punctuation))
    return txt


In [ ]:
def data_prepare(pairs):
    lang_eng = Lang('eng')
    lang_hin = Lang('hin')
    
    for pair in pairs:
        lang_eng.addSentence(pair[0])
        lang_hin.addSentence(pair[1])
    return lang_eng, lang_hin


In [ ]:
eng_train_txt = doc_reader('english.train.txt')
hin_train_txt = doc_reader('hindi.train.txt')
train_pair = make_pairs(eng_train_txt, hin_train_txt)
lang_eng, lang_hin = data_prepare(train_pair)

train_df = pd.DataFrame(train_pair)
train_df.columns = ["english", "hindi"]

list_len = []
for l in train_df.english:
    list_len.append(len(l.split(' ')))
eng_max_len = np.max(list_len)

list_len = []
for l in train_df.hindi:
    list_len.append(len(l.split(' ')))
hin_max_len = np.max(list_len)


In [ ]:
train_df


,english,hindi
0,give your application an accessibility workout,अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें
1,accerciser accessibility explorer,एक्सेर्साइसर पहुंचनीयता अन्वेषक
2,the default plugin layout for the bottom panel,निचले पटल के लिए डिफोल्ट प्लगइन खाका
3,the default plugin layout for the top panel,ऊपरी पटल के लिए डिफोल्ट प्लगइन खाका
4,a list of plugins that are disabled by default,उन प्लगइनों की सूची जिन्हें डिफोल्ट रूप से निष...
...,...,...
49995,audio test,ऑडियो जाँच
49996,silent,मूक
49997,video test,वीडियो जाँच
49998,crazy,दीवाना


In [ ]:
test_eng_text = doc_reader('english.test.txt')
test_hindi_text = doc_reader('hindi.test.txt')
test_pair = make_pairs(test_eng_text, test_hindi_text)

for pair in test_pair:
    lang_eng.addSentence(pair[0])
    lang_hin.addSentence(pair[1])

test_df = pd.DataFrame(test_pair)
test_df.columns = ["english", "hindi"]

list_len = []
for l in test_df.english:
    list_len.append(len(l.split(' ')))
eng_max_len = np.max(list_len)

list_len = []
for l in test_df.hindi:
    list_len.append(len(l.split(' ')))
hin_max_len = np.max(list_len)



In [ ]:
test_df


,english,hindi
0,a black box in your car,आपकी कार में ब्लैक बॉक्स
1,as americas road planners struggle to find the...,जबकि अमेरिका के सड़क योजनाकार ध्वस्त होते हुए ...
2,the devices which track every mile a motorist ...,यह डिवाइस जो मोटरचालक द्वारा वाहन चलाए गए प्रत...
3,the usually dull arena of highway planning has...,आम तौर पर हाईवे नियोजन जैसा उबाऊ काम भी अचानक ...
4,libertarians have joined environmental groups ...,आपने द्वारा ड्राइव किए गए मील तथा संभवतः ड्राइ...
...,...,...
2502,it is noteworthy that both nita and isha are p...,गौरतलब है कि नीता और ईशा दोनों ही प्रोफेशनल क्...
2503,vips have been invited to this royal party,इस शाही पार्टी के लिए वीवीआईपी लोगों को भी आमं...
2504,these include the jodhpur royal family and uma...,इनमें जोधपुर राजपरिवार और उम्मैद भवन के मालिक ...
2505,ln mittal sachin tendulkar and bollywood actor...,एलएनमित्तल सचिन तेंदुलकर फिल्म अभिनेता अनिल कप...


### Load Glove Vector

using 300-dim Glove word embeddings

In [ ]:
glove_emb = {}
with open('glove.840B.300d.txt') as f:
    for line in f:
        values = line.split(' ')
        # The first entry is the word
        # The rest are vectors representing the embedding for the word
        glove_emb[values[0]] = np.asarray(values[1:], dtype='float32')


--2022-04-07 11:41:36--  http://nlp.stanford.edu/data/glove.840B.300d.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.840B.300d.zip [following]
--2022-04-07 11:41:36--  https://nlp.stanford.edu/data/glove.840B.300d.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.840B.300d.zip [following]
--2022-04-07 11:41:36--  http://downloads.cs.stanford.edu/nlp/data/glove.840B.300d.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2176768927 (2.0G) [application/zip

initializing unknown tokens

words not found are assigned an unknown token


In [ ]:
UNK = np.random.random(300)
SOS = np.random.random(300)
EOS = np.random.random(300)

glove_emb['sos'] = SOS
glove_emb['eos'] = EOS

print('GloVe data loaded')


In [ ]:
emb_size = 300

vocab_size_eng = len(lang_eng.word_to_index)+1
eng_emb_matrix = np.zeros((vocab_size_eng, emb_size))
for word, i in sorted(lang_eng.word_to_index.items(), key=lambda x: x[1]):
    eng_emb_matrix[i] = glove_emb.get(word, UNK)

vocab_size_hin = len(lang_hin.word_to_index)+1
hin_emb_matrix = np.zeros((vocab_size_hin, emb_size))
for word, i in sorted(lang_hin.word_to_index.items(), key=lambda x: x[1]):
    hin_emb_matrix[i] = glove_emb.get(word, UNK)


In [ ]:

class RNN_Encoder(nn.Module):
    def __init__(self, input_size, emb_size, hidden_size, n_layers=1, bidirectional=True, batch_size=1):
        super(RNN_Encoder, self).__init__()
        self.bidirectional = bidirectional
        self.hidden_size = int(hidden_size/2)
        self.hidden_dim = self.hidden_size
        self.num_layers = n_layers
        self.batch_size = batch_size
        self.embedding = nn.Embedding.from_pretrained(
            torch.FloatTensor(eng_emb_matrix), freeze=True)
        self.lstm = nn.LSTM(emb_size, self.hidden_size,
                            n_layers*2, bidirectional=bidirectional)
        self.linear = nn.Linear(hidden_size * 2, 1)

    def forward(self, input, hidden=None):
        embedded = self.embedding(input).view(1, 1, -1)
        output, (hidden, cell) = self.lstm(embedded, hidden)
        fwd_final = hidden[0:hidden.size(0):2]
        bwd_final = hidden[1:hidden.size(0):2]
        final = torch.cat([fwd_final, bwd_final], dim=2)
        return output, final

    def initHidden(self):
        directions = 2 if self.bidirectional else 1
        return torch.zeros(
            self.num_layers*2,
            self.batch_size,
            self.hidden_size*2,
            device=device
        )


In [ ]:
class RNN_AttnDecoder(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.2, max_length=hindi_lang.n_words,n_layers=1):
        super(RNN_AttnDecoder, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p/5
        self.max_length = max_length

        self.embedding = nn.Embedding.from_pretrained(torch.FloatTensor(hin_emb_matrix), freeze=True)
        # self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn1 = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.lstm = nn.LSTM(self.hidden_size, self.hidden_size, n_layers*2)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, enc):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        intmd_torch = torch.cat((embedded[0],hidden[0]),1)
        attn_out= self.attn1(intmd_torch)
        attn_weights = F.softmax(attn_out)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 enc_ops.unsqueeze(0))
       
        output = torch.cat((embedded, attn_applied), 0)
        output = self.attn_combine(output).unsqueeze(0)
        output = F.relu(output)
        output = torch.squeeze(output,0)
        dec_hidden = self.initHidden()
        dec_cell = self.initCell()
        output1, (dec_hidden, dec_cell) = self.lstm(output, (hidden, dec_cell))
        output2 = F.log_softmax(self.out(output1[0]), dim=1)
        return output2, dec_hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1*2, 1, self.hidden_size, device=device)

    def initCell(self):
        return torch.zeros(1*2, 1, self.hidden_size, device=device)


In [ ]:

def idx_from_sent(lang, sentence):
    return [lang.word_to_index[word] for word in sentence.split(' ')]

def tensor_from_sent(lang, sentence):
    indexes = idx_from_sent(lang, sentence)
    indexes.append(token_EOS)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)

def tensor_from_pair(pair):
    return (tensor_from_sent(lang_eng, pair[0]), tensor_from_sent(lang_hin, pair[1]))



In [ ]:

tf_ratio = 1


def train(input_tensor, target_tensor, encoder, decoder, enc_op, decoder_optimizer, criterion, max_length=lang_hin.n_words):
    enc_hidden = encoder.initHidden()

    enc_op.zero_grad()
    decoder_optimizer.zero_grad()

    ip_len = input_tensor.size(0)
    target_len = target_tensor.size(0)

    enc_ops = torch.zeros(
        max_length, encoder.hidden_dim*2, device=device)

    loss = 0

    for ei in range(ip_len):
        enc_op, enc_hidden = encoder(input_tensor[ei])
        enc_ops[ei] = enc_op[0, 0]

    dec_ip = torch.tensor([[token_SOS]], device=device)

    dec_hidden = enc_hidden

    tf_flag = True if random.random() < tf_ratio else False
    # Teacher forcing: Feed the target as the next input
    # Else: use its own predictions as the next input
    if tf_flag:
        for di in range(target_len):
            dec_op, dec_hidden, decoder_attention = decoder(
                dec_ip, dec_hidden, enc_ops)
            loss += criterion(dec_op, target_tensor[di])
            dec_ip = target_tensor[di]  # Teacher forcing

    else:
        for di in range(target_len):
            dec_op, dec_hidden, decoder_attention = decoder(
                dec_ip, dec_hidden, enc_ops)
            topv, topi = dec_op.topk(1)
            # detach from history as input
            dec_ip = topi.squeeze().detach()

            loss += criterion(dec_op, target_tensor[di])
            if dec_ip.item() == token_EOS:
                break

    loss.backward()

    enc_op.step()
    decoder_optimizer.step()

    return loss.item() / target_len


In [ ]:


def train_iter(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    loss_plot = []
    tot_loss_disp = 0  # Reset every print_every
    tot_loss_plot = 0  # Reset every plot_every
    
    enc_op = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    
    random.shuffle(train_pair)
    
    criterion = nn.NLLLoss()
    
    for iter in range(0, len(train_pair)*2 + 1):
        train_pairs = tensor_from_pair(train_pair[(iter)%len(train_pair)])
        train_pair = train_pairs
        input_tensor = train_pair[0]
        target_tensor = train_pair[1]
        
        loss = train(input_tensor, target_tensor, encoder, decoder, enc_op, decoder_optimizer, criterion)
        tot_loss_disp += loss
        tot_loss_plot += loss

        if iter % print_every == 0:
            print_loss_avg = tot_loss_disp / print_every
            tot_loss_disp = 0
            print('%.4f' % (print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = tot_loss_plot / plot_every
            loss_plot.append(plot_loss_avg)
            tot_loss_plot = 0


In [ ]:


def evaluate(encoder, decoder, sentence, max_length=lang_hin.n_words):
    with torch.no_grad():
        input_tensor = tensor_from_sent(lang_eng, sentence)
        ip_len = input_tensor.size()[0]
        enc_ops = torch.zeros(max_length, encoder.hidden_dim*2, device=device)
        
        for ei in range(ip_len):
            enc_op, enc_hidden = encoder(input_tensor[ei])
            enc_ops[ei] += enc_op[0, 0]

        dec_ip = torch.tensor([[token_SOS]], device=device)
        
        dec_hidden = enc_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)
        
        for di in range(max_length):
            dec_op, dec_hidden, decoder_attention = decoder(
                dec_ip, dec_hidden, enc_ops)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = dec_op.data.topk(1)
            if topi.item() == token_EOS:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(lang_hin.index_to_word[topi.item()])

            dec_ip = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]


In [ ]:


def rand_eval(encoder, decoder, n=20):
    for i in range(n):
        pair = random.choice(test_pair)
        print('>', pair[0])
        print('=', pair[1])
        owds, attentions = evaluate(encoder, decoder, pair[0])
        op_sent = ' '.join(op_words)
        print('<', op_sent)
        print('')


In [ ]:

emb_dim = 300
hidden_size = emb_dim
hidden_dim = emb_dim
n_layers = 1

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

encoder1 = RNN_Encoder(lang_eng.n_words, emb_dim, hidden_dim, n_layers).to(device)
attn_decoder1 = RNN_AttnDecoder(emb_dim, lang_hin.n_words, n_layers).to(device)


In [ ]:
train_iter(encoder1, attn_decoder1, 100, print_every=5000)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


0.0019
5.8510
5.3962
5.3082
5.2628
5.2234
5.2242
5.1857
5.1863
5.1355
5.1481
5.1523
5.1255
5.1040
5.0991
5.0802
5.1000
5.0764
5.0903
5.0480
5.0666


In [ ]:
actual, predicted = [], []
for i in range(len(test_pair)):
    pair = test_pair[i]
    op_words, attentions = evaluate(encoder1, attn_decoder1, pair[0])
    op_sent = ' '.join(op_words)

    ls_hin = pair[1].split()
    pred_sen = op_sent.split()

    if (ls_hin[-1] == "<EOS>"):
        ls_hin = ls_hin[:-1]

    if (ls_hin[0] == "<SOS>"):
        ls_hin.pop(0)

    if (pred_sen[-1] == "<EOS>"):
        pred_sen = pred_sen[:-1]

    actual.append(ls_hin)
    predicted.append(pred_sen)

bleu_score = corpus_bleu(actual, predicted, weights=(1.0, 0, 0, 0))
print("==================================")
print("Bleu Score: ", round(bleu_score, 4))
print("==================================")


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Bleu Score:  0.3272401269530751
